# Netflix recommendation engine

Based on the [netflix prize dataset](https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data). Our
goal is to build a recommendation engine.

## Importing the libraries

In [ ]:
import polars as pl
import pandas as pd
import sqlite3

## Connect to database

Here we connect to the database `netflix_dev.db`. Currently, we are using a small portion of the whole dataset, around 100.000 / 100.000.000 Entries. This is due to the fact that the whole dataset is too big to be processed on a normal computer. We are using a sample of 100.000 entries to test our code and to get a first impression of the data. The sample is randomly chosen, so it is representative for the whole dataset.

- `netflix_data` contains the ratings from the netflix prize challenge.
- `movie_titles` contains the titles corresponding to the `film` column in `netflix_data`
- `combined` is a join of `netflix_data` and `movie_titles` over the `film` column.

In [ ]:
db = sqlite3.connect('netflix_dev.db')

netflix_data = pd.read_sql_query("SELECT * FROM netflix_data", db)
movie_titles = pd.read_sql_query("SELECT * FROM movie_titles", db)
combined     = pd.read_sql_query("SELECT * FROM netflix_data, movie_titles \
                                  WHERE netflix_data.film = movie_titles.film", db)

In [ ]:
combined

## Run some queries

Now we run some queries on the data.
- `top_100` contains the 100 most rated movies.
- `best_rated` contains the 100 best rated movies that have at least 50 ratings.
- `not_rated` contains all movies that have no ratings.

In [ ]:
top_100 = pd.read_sql_query("SELECT netflix_data.film, movie_titles.title, COUNT(*) AS 'num_ratings', AVG(netflix_data.rating) AS 'avg_rating' \
                            FROM netflix_data, movie_titles \
                            WHERE netflix_data.film = movie_titles.film \
                            GROUP BY netflix_data.film, title \
                            ORDER BY COUNT(*) DESC \
                            LIMIT 100 \
                            ", db)

top_100

In [ ]:
best_rated = pd.read_sql_query("SELECT netflix_data.film, movie_titles.title, COUNT(*) AS 'num_ratings', AVG(netflix_data.rating) AS 'avg_rating' \
                                FROM netflix_data, movie_titles \
                                WHERE netflix_data.film = movie_titles.film \
                                GROUP BY netflix_data.film, title \
                                HAVING num_ratings > 50 \
                                ORDER BY AVG(netflix_data.rating) DESC \
                                LIMIT 100", db)

best_rated

In [ ]:
not_rated = pd.read_sql_query("SELECT movie_titles.film, movie_titles.title \
                                FROM movie_titles \
                                WHERE movie_titles.film NOT IN (SELECT film FROM netflix_data)",  db)

not_rated